In [2]:
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Load Dataset.

In [3]:
url = 'https://en.wikipedia.org/wiki/World_War_II_casualties'
dfs = pd.read_html(url)

In [56]:
len(dfs)

13

# Localize CSVs

In [33]:
df = dfs[1]
df.columns = df.iloc[0]
df.drop(0).copy().to_csv('Human_losses_by_country.csv')

In [57]:
df = dfs[2]
df = df.iloc[2:8].copy()
df.columns = df.iloc[0]
df.drop(2).copy().to_csv('Third_Reich_losses.csv')

In [27]:
df = dfs[4]
df.columns = df.iloc[0]
df.drop(0).to_csv('Soviet_losses.csv')

In [33]:
df = dfs[5]
df.columns = df.iloc[0]
df.drop(0).to_csv('Holocaust_Jews.csv')

In [37]:
df = dfs[6]
df.columns = df.iloc[0]
df.drop(0).to_csv('Holocaust_Roma.csv')

In [48]:
df = dfs[7]
df.columns = df.iloc[0]
df.drop(0).to_csv('Casualties_by_Branch.csv')

In [51]:
!mkdir csvs
!mv *.csv ./csvs/
!ls
!ls csvs/

mkdir: csvs: File exists
mv: rename *.csv to ./csvs/*.csv: No such file or directory
Untitled.ipynb csvs
Casualties_by_Branch.csv    Human_losses_by_country.csv
Holocaust_Jews.csv          Soviet_losses.csv
Holocaust_Roma.csv          Third_Reich_losses.csv


# Cleaning Datasets

In [99]:
def remove_footnote(df, columns, num_footnotes):
    tmp = df.copy()
    pattern = re.compile(r'(.*)(\[.*\])(.*)')
    
    for _ in range(num_footnotes):
        for column in columns:
            tmp[column] = [pattern.sub(r'\1 \3', str(elem)) for elem in tmp[column]]
    return tmp

def remove_percentage(df, columns):
    tmp = df.copy()
    for elem in columns:
        tmp[elem] = tmp[elem].str.replace('%', '')
    return tmp

def spot_removal(df, columns):
    #Removes 'AB' from AmericaAB (note)AB
    #Removes 'A2' from (table)A2
    tmp = df.copy()
    pattern = re.compile(r'(.*[^A-Z])([A-Z]?[A-Z][0-9]?)$')
    pattern2 = re.compile(r'(.*[^A-Z])([A-Z]?[A-Z][0-9]?)(\s*\(.*\))')
    pattern3 = re.compile(r'(United Kingdom)(BE) (including Crown Colonies)')
    pattern4 = re.compile(r'(353,000)(\(1942 borders\))(  to 410,000)  ')
    for column in columns:
        tmp[column] = [pattern.sub(r'\1', str(elem)) for elem in tmp[column]]
        tmp[column] = [pattern2.sub(r'\1\3', str(elem)) for elem in tmp[column]]
        tmp[column] = [pattern3.sub(r'\1 (\3)', str(elem)) for elem in tmp[column]]
        tmp[column] = [pattern4.sub(r'\1\3', str(elem)) for elem in tmp[column]]
    return tmp

def remove_commas(df, columns):
    df_noComma = pd.DataFrame()
    for elem in columns:
        df_noComma[elem] = df[elem].str.replace(',', '')
    return df_noComma

def split_xtoy(df, columns):
    df = df.copy()
    pattern = re.compile(r'(\D*)(\d*\.?\d*)(\D*)(\d*\.?\d*)(\D*)')
    for column in columns:
        df[column+'_min'] = [pattern.sub(r'\2', str(elem)) for elem in df[column]]
        df[column+'_max'] = [pattern.sub(r'\4', str(elem)) for elem in df[column]]
    return df

### Human losses by country

In [75]:
df = pd.read_csv('./csvs/Human_losses_by_country.csv')

In [76]:
df = df.drop('Unnamed: 0', axis=1)

In [77]:
df_cleaned = remove_footnote(df, df.columns, 10)
df_cleaned = spot_removal(df_cleaned, df.columns)
df_cleaned = remove_commas(df_cleaned, df.columns)
df_minmax = split_xtoy(df_cleaned, df.columns[2:])
df_cleaned = df_minmax.replace(r'^\s*$', np.nan, regex=True) #replaces whitespace with NaN

In [78]:
df_HLBC = df_cleaned[df_cleaned.columns[:2].append(df_cleaned.columns[8:])]
df_HLBC[df_HLBC.columns[1:]] = df_HLBC[df_HLBC.columns[1:]].astype(float, copy=False)

### Third Reich

In [79]:
df = pd.read_csv('./csvs/Third_Reich_losses.csv')
df = df[df.columns[1:9]].copy()

In [80]:
df_cleaned = remove_footnote(df, df.columns, 10)
df_cleaned = spot_removal(df_cleaned, df.columns)
df_cleaned = remove_commas(df_cleaned, df.columns)
df_cleaned = df_cleaned.replace(to_replace='(1942 borders)', value='')
df_minmax = split_xtoy(df_cleaned, df.columns[2:])
df_cleaned = df_minmax.replace(r'^\s*$', np.nan, regex=True) #replaces whitespace with NaN

In [93]:
df_Axis = df_cleaned[df_cleaned.columns[:2].append(df_cleaned.columns[8:])]
df_Axis[df_Axis.columns[1:]] = df_Axis[df_Axis.columns[1:]].astype(float, copy=False)

### Soviet Union

In [174]:
df = pd.read_csv('./csvs/Soviet_losses.csv')

In [175]:
df = df.drop('Unnamed: 0', axis=1)

In [176]:
df['Deaths as\xa0% of 1940 population'] = df['Deaths as\xa0% of 1940 population'].str.replace('%', '')

In [177]:
df_Soviet = df.replace('–', np.NaN)
df_Soviet = split_xtoy(df_Soviet, df.columns[2:])
df_Soviet = df_Soviet.replace(r'^\s*$', np.nan, regex=True)

In [185]:
df_Soviet[df_Soviet.columns[1:]] = df_Soviet[df_Soviet.columns[1:]].astype(float, copy=False)
df_Soviet_final = df_Soviet[df_Soviet.columns[:2].append(df_Soviet.columns[7:])]

In [186]:
df_Soviet_final

,Soviet Republic,Population 1940 (within 1946–91 borders),Military deaths_min,Military deaths_max,Civilian deaths due tomilitary activity and crimes against humanity_min,Civilian deaths due tomilitary activity and crimes against humanity_max,Civilian deaths due towar related famine and disease_min,Civilian deaths due towar related famine and disease_max,Total_min,Total_max,Deaths as % of 1940 population_min,Deaths as % of 1940 population_max
0,Armenia,1320000.0,150000.0,NaN,NaN,NaN,30000.0,NaN,180000.0,NaN,13.6,NaN
1,Azerbaijan,3270000.0,210000.0,NaN,NaN,NaN,90000.0,NaN,300000.0,NaN,9.1,NaN
2,Belarus,9050000.0,620000.0,NaN,1360000.0,NaN,310000.0,NaN,2290000.0,NaN,25.3,NaN
3,Estonia,1050000.0,30000.0,NaN,50000.0,NaN,NaN,NaN,80000.0,NaN,7.6,NaN
4,Georgia,3610000.0,190000.0,NaN,NaN,NaN,110000.0,NaN,300000.0,NaN,8.3,NaN
5,Kazakhstan,6150000.0,310000.0,NaN,NaN,NaN,350000.0,NaN,660000.0,NaN,10.7,NaN
6,Kyrgyzstan,1530000.0,70000.0,NaN,NaN,NaN,50000.0,NaN,120000.0,NaN,7.8,NaN
7,Latvia,1890000.0,30000.0,NaN,190000.0,NaN,40000.0,NaN,260000.0,NaN,13.7,NaN
8,Lithuania,2930000.0,25000.0,NaN,275000.0,NaN,75000.0,NaN,375000.0,NaN,12.7,NaN
9,Moldova,2470000.0,50000.0,NaN,75000.0,NaN,45000.0,NaN,170000.0,NaN,6.9,NaN


# Localize Clean CSVs

In [82]:
!mkdir clean_csvs

mkdir: clean_csvs: File exists


In [83]:
df_HLBC.to_csv('./clean_csvs/Human_losses_by_country_CLEAN.csv')

In [84]:
df_Axis.to_csv('./clean_csvs/Third_Reich_losses_CLEAN.csv')

In [187]:
df_Soviet_final.to_csv('./clean_csvs/Soviet_Union_losses_CLEAN.csv')

# Creating Custom Dataset

In [234]:
!mkdir custom_csvs

## Percentage of Total Population killed by country

In [408]:
df_HLBC = pd.read_csv('./clean_csvs/Human_losses_by_country_CLEAN.csv')
df_Reich = pd.read_csv('./clean_csvs/Third_Reich_losses_CLEAN.csv')
df_Soviet = pd.read_csv('./clean_csvs/Soviet_Union_losses_CLEAN.csv')

In [409]:
HLBC = df_HLBC.iloc[:,[1, 11, 12]].copy()

In [410]:
Reich = df_Reich.iloc[:,[1, 13, 14]].copy()

In [411]:
Soviet = df_Soviet.iloc[:,[1, 11, 12]].copy()

In [412]:
columns = ['Country', '% Min', '% Max']

In [413]:
Reich.columns = columns

In [414]:
Soviet.columns = columns

In [415]:
HLBC.columns = columns

In [416]:
Custom = pd.concat([HLBC.drop([2,19, 52]),Soviet, Reich], ignore_index=True)
Custom = Custom.sort_values(by=['Country']).copy()
Custom = Custom.drop([81, 59, 13, 29, 30, 58, 76, 75, 80, 79])
Custom = Custom.reset_index(drop=True)

In [420]:
Czechoslovakia = ['Czech Republic', 'Slovak Republic']
for i in range(72, 74):
    Custom.loc[i] = Custom.loc[13].copy()
    Custom.loc[i, 'Country'] = Czechoslovakia[72 - i]
    
Yugoslavia = ['Serbia (Yugoslavia)','Montenegro (Yugoslavia)','Kosovo (Yugoslavia)', 'Croatia (Yugoslavia)', 'Slovenia (Yugoslavia)', 'Macedonia (Yugoslavia)', 'Bosnia and Herzegovina (Yugoslavia)']
for i in range(74,81):
    Custom.loc[i] = Custom.loc[71].copy()
    Custom.loc[i, 'Country'] = Yugoslavia[74 - i]

Custom.loc[60,'% Min'] = 0
Custom.loc[81] = ['Portugal', 0.0, 0.0]

In [421]:
Custom.to_csv('./custom_csvs/Death_%_population_perCountry.csv')

In [422]:
Custom

,Country,% Min,% Max
0,Albania,2.80,NaN
1,Armenia,13.60,NaN
2,Australia,0.58,NaN
3,Austria,5.56,NaN
4,Azerbaijan,9.10,NaN
5,Belarus,25.30,NaN
6,Belgium,1.05,NaN
7,Brazil,0.00,NaN
8,Bulgaria,0.33,NaN
9,Burma (British colony),1.57,NaN
